In [1]:
import xarray as xr
import rasterio
import os
import rioxarray
import gc

In [2]:
file = r'/scratch/project_2000908/nousu/mNFI_to_NEP/forest_data.nc'
data = xr.open_dataset(file)

In [3]:
subset = False

if subset:
    # 1. Get coordinate ranges
    x_coords = data['x'].values
    y_coords = data['y'].values

    # 2. Compute lengths
    x_len = len(x_coords)
    y_len = len(y_coords)

    # 3. Get 1/5th width and height
    x_width = x_len // 5
    y_height = y_len // 5

    # 4. X slice in the middle
    x_start = x_len // 2 - x_width // 2
    x_end = x_start + x_width

    # 5. Y slice around 3/4 down
    y_start = int(y_len * 3 / 4) - y_height // 2
    y_end = y_start + y_height

    # 6. Convert to coordinate slices
    x_slice = slice(x_coords[x_start], x_coords[x_end])
    y_slice = slice(y_coords[y_start], y_coords[y_end])

    # 7. Apply the slice
    data = data.sel(x=x_slice, y=y_slice)

In [3]:
pt_kivennäismaa = [1]
pt_turvemaa = [2,3,4]

kt_ravinteikas = [1,2,3]
kt_ei_ravinteikas = [4,5,6,7,8,9,10]

# Output settings
output_dir = r'/scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi'
os.makedirs(output_dir, exist_ok=True)
crs = "EPSG:3067"
nodata_value = -9999

# Process each class combination individually
for maaluokka in [1, 2, 3]:
    for paatyyppi in [1,2]:
        pt_luokat = pt_kivennäismaa if paatyyppi == 1 else pt_turvemaa
        for ravinteikkuus in [1,2]:
            kt_luokat = kt_ravinteikas if ravinteikkuus == 1 else kt_ei_ravinteikas

            mask = (
                (data['maaluokka'] == maaluokka) &
                (data['paatyyppi'].isin(pt_luokat)) &
                (data['kasvupaikka'].isin(kt_luokat))
            )

            for var in ['ika', 'ppa']:
                var_name = f"{var}_{maaluokka}_{paatyyppi}_{ravinteikkuus}"
                da = data[var].where(mask).astype('float32').fillna(nodata_value)

                # Set CRS and nodata
                da.rio.write_crs(crs, inplace=True)
                da.rio.set_nodata(nodata_value, inplace=True)

                # Save to raster
                output_path = os.path.join(output_dir, f"{var_name}.tif")
                da.rio.to_raster(output_path)

                print(f"Saved {var_name} to {output_path}")

                # Cleanup
                del da
                gc.collect()  # Free memory

Saved ika_1_1_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ika_1_1_1.tif
Saved ppa_1_1_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ppa_1_1_1.tif
Saved ika_1_1_2 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ika_1_1_2.tif
Saved ppa_1_1_2 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ppa_1_1_2.tif
Saved ika_1_2_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ika_1_2_1.tif
Saved ppa_1_2_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ppa_1_2_1.tif
Saved ika_1_2_2 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ika_1_2_2.tif
Saved ppa_1_2_2 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ppa_1_2_2.tif
Saved ika_2_1_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ika_2_1_1.tif
Saved ppa_2_1_1 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ppa_2_1_1.tif
Saved ika_2_1_2 to /scratch/project_2000908/nousu/mNFI_to_NEP/classified_mnfi/ik